In [ ]:
## For colab
## Download the specific file to the current directory
!wget https://raw.githubusercontent.com/filanova/summer_school_2026/refs/heads/main/modules.py

# to delete a file execute:
# !rm modules.py

In [ ]:
import numpy as np                 # Import NumPy
import matplotlib.pyplot as plt    # Standard library import for plots
import modules as mdl              # Specific library for this notebook (is provided in modules.py file)

!pip install findiff
from findiff import Diff           # Numerical diffferentiation


## 1. Introduction to Operator Inference
### 1.1. Continuous-time linear systems

Consider a continuous-time **Linear Time-Invariant (LTI)** system described by the state-space equation:

\begin{equation}
\tag{1}
\dot{\mathbf{x}}(t) = \mathbf{A} \mathbf{x}(t) + \mathbf{H} (\mathbf{x}(t) \otimes \mathbf{x}(t))
\end{equation}


where  
* $\mathbf{x}(t) \in \mathbb{R}^{n} \quad \;$        - state variable,
* $\mathbf{x}_0 \in \mathbb{R}^{n} \quad \quad$         - initial condition,
* $\mathbf{A} \in \mathbb{R}^{n \times n} \quad $  - system matrix (linear term),
* $\mathbf{H} \in \mathbb{R}^{n \times n^2} \quad $  - system matrix (quadratic term).

Dimension $n$ reflects the number of discretization points in the solution domain, called **degrees of freedom (DOFs)**.

We assume that snapshots of the state $\mathbf{x}(t)$ and the state derivatives $\frac{d}{dt} \mathbf{x}(t)$ are available for selected time points $t_1 , \ldots , t_k$, and collected in the snapshot matrices as follows:


$$
\mathbf{X} = \begin{bmatrix}
| & | & & | \\
\mathbf{x}(t_0) & \mathbf{x}(t_1) & \dots & \mathbf{x}(t_k) \\
| & | & & |
\end{bmatrix}, \quad
\dot{\mathbf{X}} = \begin{bmatrix}
| & | & & | \\
\frac{d}{dt}\mathbf{x}(t_0) & \frac{d}{dt}\mathbf{x}(t_1) & \dots & \frac{d}{dt}\mathbf{x}(t_k) \\
| & | & & |
\end{bmatrix}
$$

In [ ]:
# ===================================================================================================================
# PARAMETERS
# ===================================================================================================================
n       = 500                                    # domain size
mu      = 0.02                                   # viscosity
T       = 1                                      # end time
dt      = 0.01                                   # time step
inifunc = lambda x : np.sin(np.pi*x)             # initial value function

# ===================================================================================================================
# TIME-STEPPING
# ===================================================================================================================
k     = int(T/dt + 1)          # number of time points
tspan = np.linspace(0,T,k)     # time points

# ===================================================================================================================
# SIMULATION
# ===================================================================================================================
snapshots = mdl.solve_burgers(n, tspan, inifunc, mu, FOM = True)

<u>Visualization</u>

* The next cell plots different trajctories using the predifined functions from `modules.py`.

    * `visualization1d([t_point1, t_point2, ...], time_step, snapshot, kwargs)` plots the trajectories from one snapshot set at time points specified in `[t_point1, t_point2, ...]`. The trajectories `x` are the solution values at spatial domain coordinates `ξ`.

    * `visualization2d(tspan, snapshot, **kwargs)` renders the solution trajectories as a **2D** heatmap. This visualization captures the complete spatiotemporal dynamics, where the coordinates define the time `t` and spatial domain `ξ`, and the color intensity corresponds to the amplitude of the state variable `x`.

    * `visualization3d(tspan, snapshot, **kwargs)` renders the solution trajectories as a **3D** surface plot. This visualization captures the spatiotemporal evolution by mapping the time t and spatial domain `ξ` to the base axes, while the vertical axis represents the amplitude of the state variable `x`. This perspective allows for a detailed topological analysis of the system's stability and transient behavior.

<font color='blue'> **Your task** </font>

- <font color='blue'>**plot solution trajectories at different time points for different initial conditions.**</font>

In [ ]:
# ====================================================================================
# Visualization 1D
# ====================================================================================
mdl.visualize1d( [0.1, 0.9, 0.5], dt, snapshots, title="Solution snapshots")

In [ ]:
# ====================================================================================
# Visualization 2D
# ====================================================================================
mdl.visualize2d(tspan, snapshots, title="2D-plot over the domain.")


In [ ]:
# ====================================================================================
# Visualization 3D
# ====================================================================================

mdl.visualize3d(tspan, snapshots, title="3D plot")


## 2. Operator Inference Framework

- As mentioned above, our main assumption is that we only have simulation data and the system matrices are unknown. Additionally, our data is high-dimensional (number of DOFs $n$ is very large). Instead, we assume that we can represent the system dynamics with:

\begin{equation}
\tag{2}
\dot{\widehat{\mathbf{x}}}(t) = \hat{\mathbf{A}} \widehat{\mathbf{x}}(t) + \hat{\mathbf{H}} (\widehat{\mathbf{x}}(t) \otimes \widehat{\mathbf{x}}(t))
\end{equation}

$$
\boxed{\text{Our goal is to learn the reduced linear operator } \hat{\mathbf{A}} \in \mathbb{R}^{r \times r}, \text{and the reduced quadratic operator} \hat{\mathbf{H}} \in \mathbb{R}^{r \times r^2}.}
$$

- **The procedure simply contains two steps:**

    **1. Data compression,**

    **2. Optimization problem for unknown $\hat{\mathbf{A}}, \hat{\mathbf{H}}$.**


<span style="color:deepskyblue; text-decoration:underline;">**Data compression: Low-dimensional representation**</span>

- Singular value decomposition (SVD) of a matrix $\mathbf{X} \in \mathbb{R}^{n \times k}$ is defined as $$\mathbf{X} = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^{\top}$$ where
    * $\mathbf{U} \in \mathbb{R}^{n \times n}$ - orthogonal matrix ($\mathbf{U} \mathbf{U}^\top = \mathbf{U}^\top \mathbf{U} = \mathbf{I}$) containing left singular vectors (spatial modes),
    * $\mathbf{\Sigma} \in \mathbb{R}^{n \times k}$ - diagonal matrix with singular values $\sigma_{i+1} >= \sigma_i$,
    * $\mathbf{V} \in \mathbb{R}^{k \times k}$ - orthogonal matrix ($\mathbf{V} \mathbf{V}^T = \mathbf{V}^\top \mathbf{V} = \mathbf{I}$) containing right singular vectors (temporal modes).

- By using the subspace spanned by the *r* most dominant left singular vectors $\mathbf{U}_r$, we can obtain a <span style="color:deepskyblue; font-family:monospace;">low dimensional data representation </span>:$$ \mathbf{X}_r = \mathbf{U}_r^\top \mathbf{X},$$
where $\mathbf{X}_r \in \mathbb{R}^{r \times k}$.

- The information loss can be measured as $$\| \mathbf{X} - \mathbf{U}_r \mathbf{X}_r \|_2 \leq \sum_{i = r+1}^{n} \sigma_i $$

- There are different criteria for choosing the reduced dimension $r$, for example:

    * **Elbow-method** :The singular value decay is plotted in a semilogarithmic scale. The sharp bend of the curve - an "*elbow -point*" - indicates that from that dimension, the singular values have less contribution to the data and can be neglected without significant loss of the information.

    * **Cumulative energy method** : For each singular value cumulative energy is calculated: the ratio of the cumulative sum of the squared singular values to the total sum of the squared singular values.$$E_{\text{cum}} = \frac{\sum_{i=1}^{k} \sigma_i^2}{\sum_{i=1}^{n} \sigma_i^2}$$ This ratio represents the proportion of the data variance that is captured by the selected singular values. The energy method requires setting a threshold for the desired level of variance, such as 90% or 95%, and choosing the smallest number of singular values that satisfy this threshold.

    * and [many other](https://www.sciencedirect.com/science/article/pii/S2772415822000244?via%3Dihub) criteria.

<font color = blue>**Your task:**</font>


* <font color = blue>**perform an SVD decomposition for a data set**</font>
* <font color = blue>**choose a reduced order and project the data**</font>
* <font color = blue>**analyze the data approximation error**</font>


In [ ]:
# ===================================================================================================================
# Singular value decomposition
# ===================================================================================================================
U, S, V = np.linalg.svd(????, full_matrices=False) # Left singular vecs, singular vals, right singular vecs


squared_S = S**2                                           # Cumulative energy
Ecum = np.cumsum(squared_S) / np.sum(squared_S)            #


mdl.plot_svd(2,S,Ecum)                                    # Plot the singular value decay and cumulative energy
                                                          # Horizontal line denotes a hypothetical truncation order r

In [ ]:
# ===================================================================================================================
# Cumulative energy criteria to choose the reduced order
# ===================================================================================================================
tol = 0.0001                          # Energy tolerance: small threshold, denoting how much energy we can loose maximally
r_cum = np.argmax(Ecum > (1 - tol))  # Finds the first index where the condition is True. Returns 0 if the condition is never met (so check .any() if unsure)

print(f"Order of reduced model according to the cum.energy method: {r_cum}")
print(f"Energy captured by the snapshots: {100*Ecum[r_cum]}%")                 # Cumulative energy

<font color = blue>**Your task**</font>

- <font color = blue>**What does the data approximation error show? How to improve the data approximation error?**</font>
- <font color = blue>**choose the final projection basis and reduced order.**</font>


In [ ]:
# ===================================================================================================================
# Choose the reduced order
# ===================================================================================================================
sys_order = ??

In [ ]:
# ===================================================================================================================
# Data approximation error
# ===================================================================================================================
Ur = U[:,:sys_order]
mdl.visualize2d(tspan,
                abs(Ur@(Ur.T@snapshots) - snapshots),
                  contour=True,logscale=True, title = 'Data approximation error')

In [ ]:
Xr = ???  # projected state snapshots

<span style="color:deepskyblue; text-decoration:underline;">**Optimization problem**</span>

Operator inference is based on the assumption that the ROM satisfies 
\begin{equation}
\dot{\widehat{\mathbf{x}}} (t) = \hat{\mathbf{A}} \hat{\mathbf{x}}(t) + \hat{\mathbf{H}} (\hat{\mathbf{x}} \otimes \hat{\mathbf{x}}) (t),
\end{equation}
such that we can formulate a **linear** least squares problem to obtain the reduced operators:



$$ \hat{\mathbf{A}} , \hat{\mathbf{H}} = \arg \min \| \underbrace{\begin{bmatrix} \hat{\mathbf{X}}^\intercal & (\hat{\mathbf{X}} \otimes \hat{\mathbf{X}})^\intercal \end{bmatrix}}_{\mathcal{D}} \underbrace{\begin{bmatrix} \hat{\mathbf{A}}^\intercal \\ \hat{\mathbf{H}}^\intercal \end{bmatrix}}_{\mathcal{O}} - \underbrace{\dot{\hat{\mathbf{X}}}^\intercal}_{\mathcal{R}} \|_F^2 = \arg \min_{\mathcal{O}} \| \mathcal{D} \mathcal{O} - \mathcal{R} \|_F^2$$


We call $\mathcal{D}$ - the data matrix, containing the available state and input snapshots; 

$\phantom{We call}\mathcal{R}$ - the right-hand side matrix equal to the derivative snapshot matrix; 

$\phantom{We call}\mathcal{O}$ - are the unknown operators. 

<font color = blue>**Your task:**</font>

- <font color = blue>**How to solve the optimization problem?**</font>
- <font color = blue>**What is the difference to the DMD approachm?**</font>

In [ ]:
# DERIVATIVE SNAPSHOTS
# ===================================
# Approximate the state derivatives using finite differences.
#
accuracy = 4
d_dx     = Diff(1, dt, acc=accuracy)
dXr      = d_dx(Xr)

# ====================================================================================================================
# DATA MATRICES
# ====================================================================================================================
# Specify the data matrix and the right hand side for the optimization, according to (13)
#
X2r  = mdl.colwise_kron(Xr, Xr)                    # Quadratic snapshots
RHS  = dXr.T                             # Right hand side
Dmat = np.hstack([Xr.T , X2r.T])         # Data matrix

In [ ]:
# ====================================================================================================================
# OPERATOR INFERENCE
# ====================================================================================================================
def opinf_burgers(r, Dmat, RHS, tol = 1e-15):
    # r    - reduced dimension
    # Dmat - Data matrix
    # RHS  - Right hand side
    # tol  - Tolerance for the small singular values, see rcond
    
    Opmat = np.linalg.lstsq(Dmat,RHS,rcond=tol)[0].T # Unknown operator matrix np.linalg.lstsq
    
    # Assign the respective values for the reduced-order model.
    Ar = Opmat[: , :r]
    Hr = Opmat[: , r:]

    return Ar, Hr
# ===================================================================================================================
# CONSTRUCT ROM OPERATOR INFERENCE
# ===================================================================================================================
Ar, Hr = opinf_burgers(sys_order, Dmat, RHS, tol = 1e-5)



In [ ]:

# ===================================
# ROM SIMULATION
# ===================================
inivalr = Ur.T@snapshots[:,0]              # projected initial condition vector
reduced_solution     = mdl.solve_burgers(sys_order, tspan, inivalr, mu, Ar = Ar, Hr = Hr) # integration

opinf_snapshots   = Ur@reduced_solution # Transform back to the full dimension


In [ ]:
# ===================================
# ROM ANALYSIS
# ===================================
#
# Condition number
#
print('System operators \tCondition number')
print('-------------------------------------')
print(f'Ar\t{np.linalg.cond(Ar):.4e}\nHr\t{np.linalg.cond(Hr):.4e}')

In the next cell, you can plot and compare the trajectories at different time points, to see, how well the ROM performance works.

<font color = blue>**Your task:**</font>

- <font color = blue>**How else we can check how well the ROM is working? Write the list down**</font>

In [ ]:
# ====================================================================================
# Visualization 1D
# ====================================================================================
mdl.visualize1d( [0.1, 0.9, 0.5], dt, snapshots, title="FOM vs ROM simulation results", red_snapshots = opinf_snapshots)

<font color = blue>**Your task:**</font>

- <font color = blue>**Is the dimension of the current nonlinear operator $\mathbf{H}$ optimal? How to avoid redundant quadratic terms?**</font>
- <font color = blue>**Program operator inference for a linear systems $\dot{\mathbf{x}}(t) = \mathbf{A} \mathbf{x}(t)$ using the given data set!**</font>

In [ ]:
from scipy.io import loadmat       # Import function to load  .mat files
## Download the file from GitLab
## -O renames the file locally to 'data.mat' so it's easy to refer to
!wget -O data.mat "https://gitlab.mpi-magdeburg.mpg.de/goyalp/learning_linear_stablesystems/-/raw/master/data/Burger_data_init_condions.mat"

## Verify the file was downloaded
#!ls -lh data.mat

In [ ]:
# ====================================================================================
# Load data
# ====================================================================================
# data = loadmat("./data/Burger_data_init_condions.mat")        # Load data from a .mat file into a dictionary variable "data"
data = loadmat("data.mat")                                    # For colab
t     = data["t"].T                                           # Time vector [n_times x 1]
snapshots_all = data["X_data"].transpose(0, 2, 1)                     # Snapshots [n_inits x n_dofs x n_times]


n_inits, n_dofs, n_times,  = snapshots_all.shape                        # Assign the repective dimensions: number of initial conditions, number of time points, number of DOFs

dt = (t[2] - t[1]).item()                                     # Calculate the time step from the time vector

snapshots = snapshots_all[0]  # Choose a different initial condition

In [ ]:
U, S, V = np.linalg.svd(snapshots, full_matrices=False) # Left singular vecs, singular vals, right singular vecs

sys_order =  ??

Xr = ??
dXr = ??

# ====================================================================================================================
# OPERATOR INFERENCE
# ====================================================================================================================
def opinf_burgers_linear(??):
    
    Ar = ??
    

    return Ar
# ===================================================================================================================
# CONSTRUCT ROM OPERATOR INFERENCE
# ===================================================================================================================
Ar = opinf_burgers_linear(???)
                              
# ===================================
# ROM SIMULATION
# ===================================
inivalr = ??             # projected initial condition vector
reduced_solution     = mdl.solve_burgers_linear(sys_order, tspan, inivalr, Ar = Ar) # integration

opinf_snapshots   = ?? # Transform back to the full dimension
# ====================================================================================
# Visualization 1D
# ====================================================================================
mdl.visualize1d( [0.1, 0.9, 0.5], dt, snapshots, title="FOM vs ROM simulation results", red_snapshots = opinf_snapshots)